# Altair: Vega-Lite in Python

In this notebook we will learn about Vega-Lite in Python

There are multiple Vega and Vega-Lite wrappers in Python

The one we will learn about (which is also the most popular) is called `altair`

The purpose of this notebook is to introduce the core concepts of Altair

Further exploration and experimentation will be left as an exercise

> Note: we borrow heavily from the [official documentation](https://altair-viz.github.io/getting_started/overview.html) in this notebook. We strongly encourage you to review the documentation yourself for more examples and details on how to utilize altair in your workflow

In [ ]:
# uncomment the line below and evaluate this cell to install altair
# %pip install --user altair vega_datasets

## Overview

Core concepts:

- `alt.Chart`: The *container* for your chart specification. Typically all charts start with `alt.Chart(df: DataFrame)`
- `marks`: A *type* of visual element -- perhaps a line, circle, star, bar, etc.
- `encodings`: A *mapping* between the columns in your dataset and "visual encoding channels" -- perhaps x, y, color, etc.

### Example: cars scatter

In [ ]:
import altair as alt

# load a simple dataset as a pandas DataFrame
from vega_datasets import data
cars = data.cars()
cars.head()

In [ ]:
c_cars = alt.Chart(cars).mark_point().encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
).interactive()
c_cars

## Marks and Encodings

Let's dive into more detail about marks and encodings

We'll use the following dataset as an example

In [ ]:
import pandas as pd
data = pd.DataFrame(
    {
        "a": list("CCCDDDEEE"),
        "b": [2, 7, 4, 1, 2, 6, 8, 4, 7],
        "c": [1, 2, 3]*3,
    }
)

data

### Marks

The mark property is how altair tells Vega what *type* of element to draw

These are set on the `Chart` object using a method named `.mark_TYPE` where `TYPE` is the type of the mark:

In [ ]:
alt.Chart(data).mark_point()

In [ ]:
alt.Chart(data).mark_rect()

In [ ]:
c2 = alt.Chart(data).mark_circle()
print(type(c2))
c2

In these examples there is actually one mark per row in the dataset (9 marks)

However, all the marks are plotted on top of one another because we haven't specified where they should be plotted

To fix this we need to *encode* variables (columns) of our dataset as visual channels

To do this we use the `Chart.encode` method (notice that `Chart.mark_TYPE` returns `Chart`, so we can chain the `.encode` method call)

Below we'll instruct altair to map the column named `a` to the `x` channel, which controls the horizontal or x position of the mark

In [ ]:
alt.Chart(data).mark_point().encode(x="a")

In this chart we can see three distinct marks

There are actually three points at each of `C`, `D`, and `E`

To see all 9 points we also need to encode the `y` channel:

In [ ]:
c2 = (
    alt.Chart(data)
    .mark_point()
    .encode(
        x="a",
        y="b"
    )
)
c2

We can now map the `c` column to another channel...

> Note: `.encode` also returns a `Chart` so we can call `.encode` again to add more mappings

In [ ]:
c2.encode(color="c")

In [ ]:
c2.encode(size="c")

### Aggregations

When specifying the encoding for the chart, we mapped keyword arguments (like `x` and `y`) into strings

Above we used strings that mapped into column names

Altair has a mini-language for expressing other types of operations in the strings

We'll demonstrate this via examples

#### Example: Plotting mean over `a`

**Want**: Plot the mean of the values in column `b`, for each value in column `a`

Being pandas experts, we might first think to do a groupby then plot:

In [ ]:
(
    alt.Chart(data.groupby("a").mean().reset_index())
    .mark_point()
    .encode(x="a", y="b")
)

This certainly works, but we can actually let altair do the aggregation for us:

In [ ]:
c3 = (
    alt.Chart(data)
    .mark_point()
    .encode(x="a", y="average(b)")
)

c3

There are a few benefits to doing things this way:

1. The y-axis label was set to "Average of b" instead of just "b"
2. We can leverage further Altair operations that might not be as straightforward with raw pandas then altair
3. The aggregations or transformations happen in a context that is aware of the rest of the chart, allowing for other optimizations or conveniences (similar to setting y-axis title)

Our chart above did what we said we wanted, but looks a bit odd...

Usually an aggregation like an average is represented via bars instead of points

To make a bar chart instead we need to use the `rect` mark:

In [ ]:
c3.mark_bar()

Another tweak we might make to this chart would be to make a horizontal bar chart

To do this we need only swap the map for the `x` and `y` channels:

In [ ]:
c4 = (
    alt.Chart(data)
    .mark_bar()
    .encode(y="a", x="average(b)")
)

c4

### Viewing Chart JSON

The main purpose of the altair library is to make it convenient for Python users to create Vega-Lite compliant JSON specifications from pandas DataFrames

Altair can report back the JSON that it generated using the `to_json` method

In [ ]:
print(c4.to_json())

Viewing the chart JSON can be a useful debugging tool when trying to learn from the Altair or Vega-Lite documentation

## Data in Altair

Let's take a closer look at the `encoding` section of the Vega-Lite JSON for `c4` from above:

In [ ]:
print(c4.encoding.to_json())

Notice that both "x" and "y" have a `type` field

Vega-Lite requires that all encoding channels have a type

Altair took care of these for us based on the dtype of the DataFrame column

There are 5 core types of encoding, summarized in the table below:

<table class="docutils" border="1">
<colgroup>
<col width="16%">
<col width="19%">
<col width="65%">
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Data Type</th>
<th class="head">Shorthand Code</th>
<th class="head">Description</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td>quantitative</td>
<td><code class="docutils literal"><span class="pre">Q</span></code></td>
<td>a continuous real-valued quantity</td>
</tr>
<tr class="row-odd"><td>ordinal</td>
<td><code class="docutils literal"><span class="pre">O</span></code></td>
<td>a discrete ordered quantity</td>
</tr>
<tr class="row-even"><td>nominal</td>
<td><code class="docutils literal"><span class="pre">N</span></code></td>
<td>a discrete unordered category</td>
</tr>
<tr class="row-odd"><td>temporal</td>
<td><code class="docutils literal"><span class="pre">T</span></code></td>
<td>a time or date value</td>
</tr>
<tr class="row-even"><td>geojson</td>
<td><code class="docutils literal"><span class="pre">G</span></code></td>
<td>a geographic shape</td>
</tr>
</tbody>
</table>

Using the shorthand code we can give altair a hint about the type of our columns

We'll see that the type has significant implications for some encoding channels...

In [ ]:
c2.encode(color="c:Q").mark_bar()

In [ ]:
c2.encode(color="c:O").mark_bar()

In [ ]:
c2.encode(color="c:N").mark_bar()

The shorthand for specifying the type of an encoding also works when using an aggregation:

In [ ]:
(
    alt.Chart(data)
    .mark_bar()
    .encode(x="a", y="average(b):Q")
)

In [ ]:
(
    alt.Chart(data)
    .mark_bar()
    .encode(x="a", y="average(b):N")
)

In [ ]:
(
    alt.Chart(data)
    .mark_bar()
    .encode(x="a", y="average(b):O")
)

Sometimes the `keyword=STRING` shorthand isn't flexible enough for a particular application

Altair also lets you construct the encoding channels using `alt.CHANNEL` types

These types are passed as unordered positional arguments before any keyword arguments


In [ ]:
(
    alt.Chart(data)
    .mark_bar()
    .encode(
        alt.X("a"), # x = "a"  
        alt.Y("b", aggregate="average", type="quantitative"),  # y="average(b):Q"
#         alt.Color("c"),
        color="c",
    )
)

### Data from files

In addition to setting data for our charts by passing in a DataFrame, we could also pass a url to a remote dataset

> Note when not using a DataFrame, we **must** specify column types

In [ ]:
url_cars = "https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/cars.json"

c_cars_url = alt.Chart(url_cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
).interactive()

c_cars_url

In [ ]:
url_aapl = "https://raw.githubusercontent.com/plotly/datasets/master/2014_apple_stock.csv"

c_aapl = (
    alt.Chart(url_aapl)
    .mark_line()
    .encode(x="AAPL_x:T", y="AAPL_y:Q")
).interactive()

c_aapl

This is not much easier than generating a DataFrame using `pd.read_csv(url_aapl)` and then passing the DataFrame to Altair

So, why would we do it?

The benefit here is that the JSON spec for the chart can actually contain a URL which will be handled by the Vega-Lite runtime when rendering the chart

With a DataFrame, all the data is written out/hard-coded into the JSON spec before Vega-Lite sees it

In [ ]:
print(len(c_cars.to_json()))  # not going to print the whole thing... too long

In [ ]:
print(len(c_cars_url.to_json()))

In [ ]:
print(c_cars_url.to_json())

In [ ]:
print(c_cars.to_json())

The smaller spec size makes a Vega-Lite chart more suitable for sharing, loading into websites, or tracking with version control systems

### Other features

There are many other features we didn't cover:

- Chart Types: maps, candlesticks, compound chart types (multiple marks), heatmaps, area chart, scatter charts, etc...
- Compound Charts: multiple subplots in one figure
- Interactivity: linked brushing
- Customization: colors, labels

The best way to learn these concepts is by practice and study of the documentation

We'll provide opportunity for both on the upcoming homework

### Saving to webpage

The last thing we will show is how straightforward it is to include an Altair chart on a webpage

The `Chart` type has a `to_html` method that will generate an html document

This can be used directly as a standalone webpage, or parts can be copied and pasted into an existing page



In [ ]:
print(c_aapl.to_html())

In [ ]:
with open("aapl_altair_chart.html", "w") as f:
    f.write(c_aapl.to_html())